In [9]:
import pandas as pd
import numpy as np

In [51]:
encoded_data = pd.read_csv("encoded_csv.csv")
encoded_data = encoded_data.to_numpy()
print(encoded_data)

[[0.0444299  0.04119755 0.04119755 ... 0.         0.         0.        ]
 [0.04442989 0.04119757 0.04119757 ... 0.         0.         0.        ]
 [0.04442948 0.0411981  0.04119721 ... 0.         0.         0.        ]
 ...
 [0.04932845 0.0400026  0.04094935 ... 0.         0.         0.        ]
 [0.04325069 0.04307167 0.0373346  ... 0.         0.         0.        ]
 [0.04448239 0.04090727 0.04095293 ... 0.         0.         0.        ]]


In [57]:
#Penser a rajouter l'identité dans les clusters

from sklearn.cluster import KMeans

def calcCodebook(n_clusters, n_codebook): 
    
    
    kmeans = KMeans(n_clusters = n_clusters, random_state = 42)

    codebooks = []
    ecart_type = []
    labels = []
    r = encoded_data
    ecart_type.append(np.ones((n_clusters, 1)))

    for k in range(n_codebook):
        kmeans.fit(r)
        if k<n_codebook-1:
            ecart_type.append(np.expand_dims(np.array([encoded_data[kmeans.labels_==k].std() for k in range(n_clusters)]), axis=1))
        r = r-kmeans.cluster_centers_[kmeans.labels_]
        r / ecart_type[-1][kmeans.labels_]
        codebooks.append(kmeans.cluster_centers_)
        labels.append(kmeans.labels_)

    codebooks = np.array(codebooks)
    ecart_type = np.array(ecart_type)
    labels = np.array(labels)
    return (codebooks, ecart_type, labels)


In [58]:
codebooks, ecart_type, labels = calcCodebook(512, 5)

In [65]:
def calcQuantized(codebooks, ecart_type, labels):
    return np.sum(np.array([codebooks[k][labels[k]]*ecart_type[k][labels[k]] for k in range(len(codebooks))]), axis=0)

encoded_data_quantized = calcQuantized(codebooks, ecart_type, labels)
encoded_data_quantized = pd.DataFrame(encoded_data_quantized)
encoded_data_quantized.to_csv("encoded_quantized_csv.csv", index=False)